In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [33]:
#Loading the complete File

In [34]:
titanicCompleteData= pd.read_csv("C:/MyFolder/titanic.csv")

In [35]:
#In the file we have total 1310 records. From these records we have decided to keep 891 for training purpose.

In [36]:
trainData = titanicCompleteData.head(891)

In [37]:
print("Dimensions of train: {}".format(trainData.shape))


Dimensions of train: (891, 14)


In [38]:
#The reamining 418 records will be used for testing purpose.

In [39]:
untouchedTestData = titanicCompleteData.tail(418)

In [40]:
#From the test records, we are deleting the 'survived' columns.
#ONE THING IMPORTANT IS THAT WE ARE NOT DELETING FROM THE ACTUAL FILE BUT FROM THE DATAFRAME.
#ANOTHER IMPORTANT THING IS THAT- HERE WE HAVE ONE FILE WHICH WE HAVE DIVIDED INTO TWO DATAFRAMES.
#IF WE WERE GIVEN TWO FILES (TEST & TRAIN) WE COULD HAVE SIMPLY LOADED BOTH OF THEM INTO TWO DIFFERENT DATAFRAMES.
#ONE FILE | TWO FILE APPROACH MAKES NO DIFFERENCE ! 

In [41]:
del untouchedTestData['survived']

In [42]:
print("Dimensions of test: {}".format(untouchedTestData.shape))

Dimensions of test: (418, 13)


In [43]:
#Pandas has a method called 'cut'. The method is used for segmenting into categories.
#We are planning to introduce a new column 'Age Categories' in the Training set.
#We will be using 'cut' method to segment the ages into defined categories.
#EXAMPLE:
#-1 - 0 = Missing
#0 - 5 = Infant
#5 - 12 = Child
#12 - 18 = Teenager
#18 - 35 = Young Adult
#35 - 60 = Adult
#60 - 100 = Senior

In [44]:
def process_age(df,cut_points,label_names):
    df["age"] = df["age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["age"],cut_points,labels=label_names)
    return df


In [45]:
cut_points = [-1,0,5,12,18,35,60,100]
label_names = ["Missing","Infant","Child","Teenager","Young Adult","Adult","Senior"]

trainData = process_age(trainData,cut_points,label_names)
untouchedTestData = process_age(untouchedTestData,cut_points,label_names)

C:\Users\rvaish\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rvaish\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
#Notice that after the addition of new column 'Age Categories' the dimention changed from (891,14) ---> (891,15)
#AND (891,13) ---> (418,14)


In [47]:
print("Dimensions of train: {}".format(trainData.shape))

Dimensions of train: (891, 15)


In [49]:
print("Dimensions of train: {}".format(untouchedTestData.shape))

Dimensions of train: (418, 14)


In [28]:
# Right now our Training Dataframe is of (891,15) dimention.
# and our Testing Dataframe is of (418, 14) dimention.

#Now, we are going introducing 7+3 columns (in both Training and Testing Dataframes)
#7 new columns will be for ages. Default values = 0  
#3 new columns will be for class.Default values = 0  

In [51]:
new_age_columns = ["Missing","Infant","Child","Teenager","Young Adult","Adult","Senior"]

for newCol in new_age_columns:
    trainData[newCol] =0
    
for newCol in new_age_columns:
    untouchedTestData[newCol] =0
    
    
new_class_columns = ["PClass1", "PClass2", "PClass3"]
for newCCol in new_class_columns:
    trainData[newCCol] =0
    
for newCCol in new_class_columns:
    untouchedTestData[newCCol] =0

C:\Users\rvaish\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\rvaish\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\rvaish\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [52]:
print("Dimensions of train: {}".format(trainData.shape))

Dimensions of train: (891, 25)


In [60]:
print("Dimensions of train: {}".format(untouchedTestData.shape))

Dimensions of train: (418, 24)


In [61]:
# In the mission of creating sparse matrix, we will also need to change the male-female as 1 and 0 (In both the Dataframes)

In [62]:
trainData = trainData.replace({'sex':{'male':1,'female':0}})
untouchedTestData = untouchedTestData.replace({'sex':{'male':1,'female':0}})

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [63]:
#Now as for 7+3 newly created columns we have assigned 0 as default values ; but that's not we are going to take
#things forward. We need to assign respective values into the same.
#Explanation:
#In Training set, if the value of column PClass = 1 then , for mark newly created PClass1 column value as 1, and rest 0.
#Similarly
#If the value of column PClass = 3 then , for mark newly created PClass3 column value as 1, and rest 0.
#...
#...
#...


#This is Sparse Matrix Way!

In [64]:
    
trainData['PClass1'] = np.where(trainData['pclass']==1, '1', '0')
trainData['PClass2'] = np.where(trainData['pclass']==2, '1', '0')
trainData['PClass3'] = np.where(trainData['pclass']==3, '1', '0')

trainData['Missing'] = np.where(trainData['Age_categories']=="Missing", '1', '0')
trainData['Infant'] = np.where(trainData['Age_categories']=='Infant', '1', '0')
trainData['Child'] = np.where(trainData['Age_categories']=='Child', '1', '0')
trainData['Teenager'] = np.where(trainData['Age_categories']=='Teenager', '1', '0')
trainData['Young Adult'] = np.where(trainData['Age_categories']=='Young Adult', '1', '0')
trainData['Adult'] = np.where(trainData['Age_categories']=='Adult', '1', '0')
trainData['Senior'] = np.where(trainData['Age_categories']=='Senior', '1', '0')

#=======================================================================================================

untouchedTestData['PClass1'] = np.where(untouchedTestData['pclass']==1, '1', '0')
untouchedTestData['PClass2'] = np.where(untouchedTestData['pclass']==2, '1', '0')
untouchedTestData['PClass3'] = np.where(untouchedTestData['pclass']==3, '1', '0')

untouchedTestData['Missing'] = np.where(untouchedTestData['Age_categories']=="Missing", '1', '0')
untouchedTestData['Infant'] = np.where(untouchedTestData['Age_categories']=='Infant', '1', '0')
untouchedTestData['Child'] = np.where(untouchedTestData['Age_categories']=='Child', '1', '0')
untouchedTestData['Teenager'] = np.where(untouchedTestData['Age_categories']=='Teenager', '1', '0')
untouchedTestData['Young Adult'] = np.where(untouchedTestData['Age_categories']=='Young Adult', '1', '0')
untouchedTestData['Adult'] = np.where(untouchedTestData['Age_categories']=='Adult', '1', '0')
untouchedTestData['Senior'] = np.where(untouchedTestData['Age_categories']=='Senior', '1', '0')


In [65]:
trainData.head(12)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,...,Missing,Infant,Child,Teenager,Young Adult,Adult,Senior,PClass1,PClass2,PClass3
0,1,1,"Allen, Miss. Elisabeth Walton",0,29.0000,0,0,24160,211.3375,B5,...,0,0,0,0,1,0,0,1,0,0
1,1,1,"Allison, Master. Hudson Trevor",1,0.9167,1,2,113781,151.5500,C22 C26,...,0,1,0,0,0,0,0,1,0,0
2,1,0,"Allison, Miss. Helen Loraine",0,2.0000,1,2,113781,151.5500,C22 C26,...,0,1,0,0,0,0,0,1,0,0
3,1,0,"Allison, Mr. Hudson Joshua Creighton",1,30.0000,1,2,113781,151.5500,C22 C26,...,0,0,0,0,1,0,0,1,0,0
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",0,25.0000,1,2,113781,151.5500,C22 C26,...,0,0,0,0,1,0,0,1,0,0
5,1,1,"Anderson, Mr. Harry",1,48.0000,0,0,19952,26.5500,E12,...,0,0,0,0,0,1,0,1,0,0
6,1,1,"Andrews, Miss. Kornelia Theodosia",0,63.0000,1,0,13502,77.9583,D7,...,0,0,0,0,0,0,1,1,0,0
7,1,0,"Andrews, Mr. Thomas Jr",1,39.0000,0,0,112050,0.0000,A36,...,0,0,0,0,0,1,0,1,0,0
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",0,53.0000,2,0,11769,51.4792,C101,...,0,0,0,0,0,1,0,1,0,0
9,1,0,"Artagaveytia, Mr. Ramon",1,71.0000,0,0,PC 17609,49.5042,NaN,...,0,0,0,0,0,0,1,1,0,0


In [66]:
#Now choose the important variables (columns)

In [67]:
selectdColumns = ["sex","Missing","Infant","Child","Teenager","Young Adult","Adult","Senior","PClass1", "PClass2", "PClass3"]

In [ ]:
# Now we have Sparse Matrix is prepared, time to apply ML modeling.

In [75]:
#lr = LogisticRegression()

In [76]:
#lr.fit(trainData[selectdColumns], trainData["survived"])

In [68]:
rf = RandomForestClassifier (n_estimators=100)

In [268]:
rf.fit(trainData[selectdColumns], trainData["survived"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
#IMPORTANT: NOW WE ARE SPLITTING OUR TRAIN DATAFRAME IN 80% 20% RATIO.

In [78]:
all_X = trainData[selectdColumns]
all_y = trainData['survived']

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.20,random_state=0)

rf.fit(train_X, train_y)
predictions = rf.predict(test_X)
print(predictions)
accuracy = accuracy_score(test_y, predictions)
print(accuracy)

[0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0
 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
0.8156424581005587


In [ ]:
#10 FOLD CROSS VALIDATION- Therefore, 10 results in the List.
#Post that, taking mean of the calculated scores.

In [79]:
rf = RandomForestClassifier (n_estimators=100)
scores = cross_val_score(rf, all_X, all_y, cv=10)
scores.sort()
accuracy = scores.mean()

print(scores)
print(accuracy)

[0.56818182 0.61363636 0.75555556 0.77777778 0.80898876 0.80898876
 0.82222222 0.83333333 0.86363636 0.88764045]
0.7739961411871523


In [271]:
############  NOW TIME TO USE UNSEEN DATA #################

In [272]:
holdout = untouchedTestData 

In [273]:
holdout[columns]

,sex,Missing,Infant,Child,Teenager,Young Adult,Adult,Senior,PClass1,PClass2,PClass3
891,1,0,0,0,0,1,0,0,0,0,1
892,1,0,0,0,0,1,0,0,0,0,1
893,1,0,0,0,0,1,0,0,0,0,1
894,1,0,1,0,0,0,0,0,0,0,1
895,0,0,1,0,0,0,0,0,0,0,1
896,1,0,0,0,0,0,1,0,0,0,1
897,1,0,0,0,0,1,0,0,0,0,1
898,1,0,0,0,0,1,0,0,0,0,1
899,0,0,0,0,0,1,0,0,0,0,1
900,1,1,0,0,0,0,0,0,0,0,1


In [274]:
rf = RandomForestClassifier (n_estimators=100)
rf.fit(all_X,all_y)
predictions = rf.predict(holdout[columns])


In [275]:
print(predictions)

[0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0]


In [ ]:
#Now we have predictions. And from the CSV File we have the actual outputs.
#Let's assign the actual outputs from the CSV into cheatvalues

In [280]:
cheatvalues = [0,
0,
0,
1,
1,
0,
0,
0,
1,
0,
0,
0,
0,
0,
1,
0,
0,
0,
1,
0,
0,
0,
1,
0,
0,
1,
1,
0,
0,
0,
0,
1,
1,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
1,
1,
1,
0,
0,
0,
0,
1,
0,
0,
0,
1,
0,
1,
0,
1,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
1,
0,
1,
1,
0,
0,
1,
1,
0,
0,
0,
0,
0,
1,
0,
1,
0,
0,
0,
1,
0,
1,
1,
1,
1,
1,
1,
0,
1,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
1,
0,
1,
1,
0,
1,
0,
0,
0,
0,
1,
1,
1,
1,
1,
0,
1,
1,
0,
1,
1,
1,
0,
0,
1,
1,
1,
1,
0,
0,
0,
0,
0,
1,
1,
0,
0,
1,
1,
0,
0,
1,
0,
0,
1,
0,
0,
0,
1,
0,
0,
0,
0,
0,
1,
1,
1,
1,
1,
1,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
1,
1,
1,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
1,
1,
1,
1,
0,
0,
0,
0,
0,
1,
0,
1,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
1,
0,
0,
0,
1,
1,
0,
0,
1,
0,
1,
0,
1,
0,
0,
0,
1,
0,
0,
1,
0,
0,
0,
0,
0,
0,
1,
0,
1,
1,
1,
0,
1,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
1,
0,
0,
0,
0,
0,
0,
0,
0]

In [281]:
len(predictions)

418

In [282]:
len(cheatvalues)

418

In [283]:
accuracy = accuracy_score(cheatvalues, predictions)

In [284]:
print(accuracy)

0.7296650717703349


In [ ]:
#THE END 